<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#CTR-prediction" data-toc-modified-id="CTR-prediction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>CTR-prediction</a></span><ul class="toc-item"><li><span><a href="#Problem-Formulation" data-toc-modified-id="Problem-Formulation-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Problem Formulation</a></span></li><li><span><a href="#Dataset" data-toc-modified-id="Dataset-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Dataset</a></span><ul class="toc-item"><li><span><a href="#Dataset-construction:" data-toc-modified-id="Dataset-construction:-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Dataset construction:</a></span></li><li><span><a href="#Format:" data-toc-modified-id="Format:-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Format:</a></span></li></ul></li><li><span><a href="#Metrics" data-toc-modified-id="Metrics-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Metrics</a></span></li></ul></li><li><span><a href="#Dataset-preprocessing" data-toc-modified-id="Dataset-preprocessing-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Dataset preprocessing</a></span><ul class="toc-item"><li><span><a href="#ML-Pipelines-(Transformers,-Estimators)" data-toc-modified-id="ML-Pipelines-(Transformers,-Estimators)-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span><a href="https://spark.apache.org/docs/latest/ml-pipeline.html#pipeline-components" target="_blank">ML Pipelines (Transformers, Estimators)</a></a></span><ul class="toc-item"><li><span><a href="#Prepare-stages-of-pipeline" data-toc-modified-id="Prepare-stages-of-pipeline-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Prepare stages of pipeline</a></span></li><li><span><a href="#Fit-and-save-pipeline" data-toc-modified-id="Fit-and-save-pipeline-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>Fit and save pipeline</a></span></li><li><span><a href="#Load-fitted-pipeline" data-toc-modified-id="Load-fitted-pipeline-2.1.3"><span class="toc-item-num">2.1.3&nbsp;&nbsp;</span>Load fitted pipeline</a></span></li><li><span><a href="#Transform-dataset-using-pipeline" data-toc-modified-id="Transform-dataset-using-pipeline-2.1.4"><span class="toc-item-num">2.1.4&nbsp;&nbsp;</span>Transform dataset using pipeline</a></span></li><li><span><a href="#Make-dataset-split" data-toc-modified-id="Make-dataset-split-2.1.5"><span class="toc-item-num">2.1.5&nbsp;&nbsp;</span>Make dataset split</a></span></li></ul></li></ul></li><li><span><a href="#Classification" data-toc-modified-id="Classification-3"><span class="toc-item-num">3&nbsp;&nbsp;</span><a href="https://spark.apache.org/docs/latest/ml-classification-regression.html" target="_blank">Classification</a></a></span><ul class="toc-item"><li><span><a href="#Logistic-Regression" data-toc-modified-id="Logistic-Regression-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span><a href="https://spark.apache.org/docs/latest/ml-classification-regression.html#logistic-regression" target="_blank">Logistic Regression</a></a></span><ul class="toc-item"><li><span><a href="#Define-and-Train-model" data-toc-modified-id="Define-and-Train-model-3.1.1"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span>Define and Train model</a></span></li></ul></li></ul></li><li><span><a href="#Evaluation" data-toc-modified-id="Evaluation-4"><span class="toc-item-num">4&nbsp;&nbsp;</span><a href="https://spark.apache.org/docs/2.1.0/mllib-evaluation-metrics.html" target="_blank">Evaluation</a></a></span><ul class="toc-item"><li><span><a href="#Binary-classification-metrics" data-toc-modified-id="Binary-classification-metrics-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span><a href="https://spark.apache.org/docs/2.1.0/mllib-evaluation-metrics.html#binary-classification" target="_blank">Binary classification metrics</a></a></span></li><li><span><a href="#Make-submission" data-toc-modified-id="Make-submission-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Make submission</a></span></li></ul></li></ul></div>

# CTR-prediction

## Problem Formulation

$\newcommand{\vecw}{{\bf w}}$
$\newcommand{\vecx}{{\bf x}}$

* Dataset: $X^N = \{ z_i \}^N_{i=1}$, где $z_i = (\vecx_i, y_i) \sim P(z), y_i \in \{0,1\}$
* Prediction: $$ \hat{y}_i = f_{\vecw}(\vecx_i) =  \mathbb{P} \left\{ y = 1 \mid \vecx_i \right\} $$
* Loss function (Binary Cross-Entropy): $$ \min\limits_{\vecw} \quad \frac{\lambda}{2}\| \vecw \|^2_2 - \frac{1}{N} \sum\limits_{i=1}^{N} y_i \log \hat{y}_i + (1-y_i) \log(1-\hat{y}_i) $$

## Dataset
$ $
<details>
  <summary>Click here to see the details</summary>

For more details see `/data/criteo/readme.txt`

### Dataset construction:


>There are 13 features taking **integer** values and 26
**categorical** features. The values of the categorical features have been hashed
onto 32 bits for anonymization purposes. 
Some features may have missing values.

> The rows are chronologically ordered by `id` column.

> The test set corresponds to events on the day following the training period. 
The first column (`label`) has been removed.


### Format:

> The columns are comma separeted with the following schema:
`<label>,<integer feature 1>, ... <integer feature 13>,<categorical feature 1>, ... <categorical feature 26>,<id>`

> When a value is missing, the field is "". There is no `label` field in the test set.

</details>
    
## Metrics

The evaluation metrics for this task are
* ROC AUC
* LogLoss
* [Normalized Entropy](https://quinonero.net/Publications/predicting-clicks-facebook.pdf)

In [1]:
%matplotlib inline
%config InlineBackend.figure_format ='retina'

import os
import sys
import glob
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

import pyspark
import pyspark.sql.functions as F
from pyspark.conf import SparkConf
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.sql import Row


spark = SparkSession \
    .builder \
    .master('local[*]') \
    .appName("spark_sql_examples") \
    .config("spark.executor.memory", "6g") \
    .getOrCreate()

sc = spark.sparkContext
sqlContext = SQLContext(sc)

In [2]:
DATA_PATH = '/workspace/data/criteo'

TRAIN_PATH = os.path.join(DATA_PATH, 'train.csv')

Lets begin our introduction to Spark [MLlib](https://spark.apache.org/docs/latest/ml-guide.html)

---
# Dataset preprocessing

Before we can train any prediction model on our dataset we need to conver each row into real-valued features vector ($\vecx \in \mathbb{R}^n$).

Spark MLlib provides easy to use tools for preprocessing raw features and turning them into suitable format.

In [3]:
df = sqlContext.read.format("com.databricks.spark.csv") \
    .option("delimiter", ",") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load('file:///' + TRAIN_PATH)

In [4]:
df = df.sample(False, 0.1)

Keeping only first two categorical features for simplicity

In [5]:
num_columns = ['_c{}'.format(i) for i in range(1, 14)]
cat_columns = ['_c{}'.format(i) for i in range(14, 40)][:2]
len(num_columns), len(cat_columns)

(13, 2)

In [6]:
df = df.fillna(0, subset=num_columns)

In [7]:
cnt_feats = df.columns[1:14]

hash_feats = df.columns[14:-1]

label = df.columns[0]

## [ML Pipelines (Transformers, Estimators)](https://spark.apache.org/docs/latest/ml-pipeline.html#pipeline-components)


MLlib standardizes APIs for machine learning algorithms to make it easier to combine multiple algorithms into a single pipeline, or workflow.

* `Transformer`: A Transformer is an algorithm which can transform one DataFrame into another DataFrame. E.g., an ML model is a Transformer which transforms a DataFrame with features into a DataFrame with predictions.


* `Estimator`: An Estimator is an algorithm which can be fit on a DataFrame to produce a Transformer. E.g., a learning algorithm is an Estimator which trains on a DataFrame and produces a model.


* `Pipeline`: A Pipeline chains multiple Transformers and Estimators together to specify an ML workflow.

---
Basically speaking `transformer` is an instance of class that implements `transform` method, and both `estimator` and `pipeline` implements `transform` and `fit` methods.

---

### Prepare stages of pipeline

We might benefit from using `StringIndexer, OneHotEncoderEstimator, VectorAssembler` (see [doc](https://spark.apache.org/docs/latest/ml-features) for details) 

The concept behind String Indexing is very intuitive. We simply replace each category with a number. 

In [8]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoderEstimator, VectorAssembler, StandardScaler

In [9]:
indexers = [StringIndexer(inputCol=col, outputCol=col+'_index') for col in hash_feats]

ohe = OneHotEncoderEstimator(
    inputCols=[col+'_index' for col in hash_feats[:3]], 
    outputCols=[col+'_ohe' for col in hash_feats[:3]]
)

ohe_assembler = VectorAssembler(inputCols=[col+'_ohe' for col in hash_feats[:3]], outputCol='ohe_features')

num_assembler = VectorAssembler(inputCols=cnt_feats, outputCol="num_features")

scaler = StandardScaler(inputCol="num_features", outputCol="scaled_features")

final_assembler = VectorAssembler(inputCols=['scaled_features', 'ohe_features'], outputCol='features')

### Fit and save pipeline

In [10]:
pipeline  = Pipeline(stages=indexers + [ohe, ohe_assembler, num_assembler, scaler, final_assembler])

In [11]:
df_tranformed = pipeline.fit(df).transform(df).select(label, 'id', 'features')

In [13]:
qwe = df_tranformed.toPandas()

Py4JJavaError: An error occurred while calling o1638.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 11 in stage 58.0 failed 1 times, most recent failure: Lost task 11.0 in stage 58.0 (TID 679, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$9: (string) => double)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$13$$anon$1.hasNext(WholeStageCodegenExec.scala:636)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$2.apply(SparkPlan.scala:255)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$2.apply(SparkPlan.scala:247)
	at org.apache.spark.rdd.RDD$$anonfun$mapPartitionsInternal$1$$anonfun$apply$24.apply(RDD.scala:836)
	at org.apache.spark.rdd.RDD$

In [12]:
df_tranformed.write.parquet("df_tranformed")

Py4JJavaError: An error occurred while calling o1639.parquet.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:198)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:159)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:104)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:102)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:122)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:131)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:127)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:155)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScop

In [12]:
df_tranformed

DataFrame[_c0: int, id: bigint, features: vector]

In [13]:
df_tranformed.count()

366833

In [14]:
df_tranformed.cache()

DataFrame[_c0: int, id: bigint, features: vector]

In [15]:
pipeline.save("/pipeline_1")

In [17]:
df_tranformed.show(2)

+---+---+--------------------+
|_c0| id|            features|
+---+---+--------------------+
|  0|151|(149348,[1,2,3,4,...|
|  0|199|(149348,[1,2,4,7,...|
+---+---+--------------------+
only showing top 2 rows



### Load fitted pipeline

In [17]:
from pyspark.ml import PipelineModel


pipe = Pipeline.load("/pipeline_1")

### Transform dataset using pipeline

In [18]:
pipeline.transform()

AttributeError: 'Pipeline' object has no attribute 'transform'

In [16]:
pipe.transform(df)

NameError: name 'pipe' is not defined

In [20]:
pipe.stages

Param(parent='Pipeline_fe060646392c', name='stages', doc='a list of pipeline stages')

### Make dataset split

Spark provides [randomSplit](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.randomSplit) method.

It is not the best choice in our task since we have chronological order in data.

We need to implement our own split function which will split the data in parts with respect to chronological order.

In [21]:
def split_by_col(df, split_col, parts_fractions, N):
    """
    df - DataFrame
    split_col - total order column
    parts_fractions - fractions of resulting parts
    """
    
    df = df.sort(split_col, ascending=True)
    
    rdd_df = df.select(split_col).rdd.zipWithIndex()
    df_final = rdd_df.toDF()
    df_final = df_final\
        .withColumn(split_col, df_final['_1'].getItem(split_col))\
        .drop('_1')
    
    df_final = df.join(
                        df_final.withColumnRenamed('_1', 'id'),
                        how='left',
                        on=split_col
                        )
    
    train, val = round(parts_fractions[0] * N), round(sum(parts_fractions[:2]) * N)

    
    parts = []
    
    df_final = df_final.sort(split_col, ascending=True)\
        .withColumnRenamed('_2', 'idx_row')\
        .withColumn('fractions', 
                    F.when(F.col('idx_row') < F.lit(train), 'train')\
                        .otherwise(F.when((F.col('idx_row') >= F.lit(train)) & (F.col('idx_row') < F.lit(val)), 'val')\
                                   .otherwise('test'))
                   )
    parts = [df_final\
             .filter(F.col('fractions') == F.lit(frac)) for frac in ['train', 'val', 'test']]
    
    return parts

In [22]:
N = df_tranformed.count()

Py4JJavaError: An error occurred while calling o1638.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 5 in stage 123.0 failed 1 times, most recent failure: Lost task 5.0 in stage 123.0 (TID 738, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$9: (string) => double)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$13$$anon$1.hasNext(WholeStageCodegenExec.scala:636)
	at org.apache.spark.sql.execution.columnar.CachedRDDBuilder$$anonfun$1$$anon$1.hasNext(InMemoryRelation.scala:125)
	at org.apache.spark.storage.memory.MemoryStore.putIterator(MemoryStore.scala:221)
	at org.apache.spark.storage.memory.MemoryStore.putIteratorAsValues(MemoryStore.scala:299)
	at org.apache.s

In [121]:
train_df, val_df, test_df = split_by_col(data_2.select('id', 'features', label), 'id', [0.8, 0.1, 0.1], N)

In [122]:
train_df.count() / N, val_df.count() / N, test_df.count() / N

(0.7999997816996646, 0.10000010915016772, 0.10000010915016772)

In [111]:
x = '(0.7999997816996646, 0.10000010915016772, 0.10000010915016772)'
sum(map(float, x.strip('(').strip(')').split(', ')))

1.0

In [19]:
train_df.count() / N, val_df.count() / N, test_df.count() / N

(0.7999996726948992, 0.09999989089829976, 0.09999989089829976)

In [112]:
x = '(0.7999996726948992, 0.09999989089829976, 0.09999989089829976)'
sum(map(float, x.strip('(').strip(')').split(', ')))

0.9999994544914987

1 min to split by col

In [123]:
train_df.cache()
val_df.cache()
test_df.cache()

DataFrame[id: bigint, features: vector, _c0: int, idx_row: bigint, fractions: string]

---
# [Classification](https://spark.apache.org/docs/latest/ml-classification-regression.html)

## [Logistic Regression](https://spark.apache.org/docs/latest/ml-classification-regression.html#logistic-regression)

### Define and Train model

In [20]:
from pyspark.ml.classification import LogisticRegression 
lr_model = LogisticRegression(featuresCol='features', labelCol=label)

In [23]:
df_tranformed

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1159, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 985, in send_command
    response = connection.send_command(command)
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1164, in send_command
    "Error while receiving", e, proto.ERROR_ON_RECEIVE)
py4j.protocol.Py4JNetworkError: Error while receiving
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:44843)
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", 

Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:44843)

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:44843)
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1067, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused


In [22]:
sample = df_tranformed.sample(0.5)

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1159, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 985, in send_command
    response = connection.send_command(command)
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1164, in send_command
    "Error while receiving", e, proto.ERROR_ON_RECEIVE)
py4j.protocol.Py4JNetworkError: Error while receiving


Py4JError: An error occurred while calling o1833.sample

In [21]:
lr_pred = lr_model.fit(df_tranformed).transform(df_tranformed)

Py4JJavaError: An error occurred while calling o2549.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 4 in stage 164.0 failed 1 times, most recent failure: Lost task 4.0 in stage 164.0 (TID 877, localhost, executor driver): java.lang.OutOfMemoryError: GC overhead limit exceeded

Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:1889)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1877)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1876)
	at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:48)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:1876)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleTaskSetFailed$1.apply(DAGScheduler.scala:926)


----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 60488)
----------------------------------------


Traceback (most recent call last):
  File "/usr/lib64/python3.5/socketserver.py", line 313, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib64/python3.5/socketserver.py", line 341, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib64/python3.5/socketserver.py", line 354, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib64/python3.5/socketserver.py", line 681, in __init__
    self.handle()
  File "/usr/local/spark/python/pyspark/accumulators.py", line 269, in handle
    poll(accum_updates)
  File "/usr/local/spark/python/pyspark/accumulators.py", line 241, in poll
    if func():
  File "/usr/local/spark/python/pyspark/accumulators.py", line 245, in accum_updates
    num_updates = read_int(self.rfile)
  File "/usr/local/spark/python/pyspark/serializers.py", line 717, in read_int
    raise EOFError
EOFError


In [ ]:
11-43

In [124]:
from pyspark.ml.classification import LogisticRegression 


lr_model = LogisticRegression(featuresCol='features', labelCol=label)

In [4]:
from pyspark.ml.classification import LogisticRegression 


lr_model = LogisticRegression(featuresCol='features', labelCol=label)

In [ ]:
lr_model.t

In [131]:
from pyspark.ml import classification as clf


lr_model2 = clf.LogisticRegression(featuresCol='features', labelCol=label)

In [125]:
lr_model.fit(train_df)

LogisticRegressionModel: uid = LogisticRegression_18648c0926ad, numClasses = 2, numFeatures = 1406

In [132]:
lr_model2.fit(train_df)

LogisticRegressionModel: uid = LogisticRegression_f909d9daa496, numClasses = 2, numFeatures = 1406

In [ ]:
lr_model2.t

In [133]:
lr_model.probabilityCol

Param(parent='LogisticRegression_18648c0926ad', name='probabilityCol', doc='Column name for predicted class conditional probabilities. Note: Not all models output well-calibrated probability estimates! These probabilities should be treated as confidences, not precise probabilities.')

In [127]:
lr_model.predict(val_df).first()

AttributeError: 'LogisticRegression' object has no attribute 'predict'

---
# [Evaluation](https://spark.apache.org/docs/2.1.0/mllib-evaluation-metrics.html)

## [Binary classification metrics](https://spark.apache.org/docs/2.1.0/mllib-evaluation-metrics.html#binary-classification)

* ROC AUC
* LogLoss
* Normalized Entropy

In [22]:
from pyspark.sql.types import *
from pyspark.mllib.evaluation import BinaryClassificationMetrics


def rocauc(model, df):
    ######################################
    ######### YOUR CODE HERE #############
    ######################################
    pass


def logloss(model, df):
    ######################################
    ######### YOUR CODE HERE #############
    ######################################
    pass


def ne(model, df):
    ######################################
    ######### YOUR CODE HERE #############
    ######################################
    pass

In [23]:
rocauc(lr_model, val_df)

0.7031266528628317

In [24]:
ne(lr_model, val_df)

0.918952248522235

In [25]:
rocauc(lr_model, test_df)

0.7000027547230876

In [26]:
ne(lr_model, test_df)

0.9220781622206322

## Make submission

Join the [competition](https://www.kaggle.com/c/mlbd-20-ctr-prediction-1) and make a submission

In [27]:
######################################
######### YOUR CODE HERE #############
######################################